In [73]:
import sys
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import spacy
from allennlp.predictors.predictor import Predictor

%matplotlib inline

In [74]:
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")

# Count metrics

In [75]:
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
def list2set(x):
    return set(x)

In [76]:
test_df =pd.read_csv('event2mind/test.csv')[['Event','Xintent','Xemotion','Otheremotion']]

### Uniqe Events

In [77]:
def uniq_events (test_df):
    test_df["Xintent"] = test_df["Xintent"].apply(str2list)
    test_df["Xemotion"] = test_df["Xemotion"].apply(str2list)
    test_df["Otheremotion"] = test_df["Otheremotion"].apply(str2list)
    test_df = test_df.groupby(["Event"])["Xintent", "Xemotion", "Otheremotion"].sum()
    
    test_df["Xintent"] = test_df["Xintent"].apply(list2set)
    test_df["Xemotion"] = test_df["Xemotion"].apply(list2set)
    test_df["Otheremotion"] = test_df["Otheremotion"].apply(list2set)
    return test_df

In [78]:
test_df = uniq_events(test_df)

In [79]:
test_df.head()

,Xintent,Xemotion,Otheremotion
Event,,,
It always hated ___,{none},{none},"{annoyed, displeasured, anger, frustration, fr..."
It cries and cried,{none},{none},"{sorrow, sad, depresed, grief, sadness}"
It does n't last long,{none},{none},{sad}
It does not go well,{none},{none},"{disappointing, disappointed, embarassment, an..."
It falls open,{none},{none},"{curious, judgemental, surprised, startled}"


### recall@10

In [80]:
def check_metrics (column_name, data_set, kill_none = False):
    connected = {"Xintent": "xintent", "Xemotion": "xreact", "Otheremotion": "oreact"}
    
    average = 0
    L = data_set.shape[0]
    counter = 0; n = 0

    
    for index, row in data_set.iterrows():
        if(n == (counter*100)//L):
            print("\rCalculating: " + str(n) + "%", end = "")
            n += 1
            
        result = predictor.predict(source=index)
        model_out = [ (" ".join(react), prob) for react, prob in zip(result[connected[column_name] + "_top_k_predicted_tokens"], result[connected[column_name] + "_top_k_log_probabilities"])]
        test_set = row[column_name]
        model_set = set()
        for word in model_out[:10]:
            model_set.add(word[0])
        if(kill_none == True):
            if("none" in test_set):
                test_set.remove("none")
            if("none" in model_set):
                model_set.remove("none")
        if(len(test_set) != 0):
            average += len(test_set & model_set) / len(test_set)
        counter += 1
        
    print("\r", end = "")
    return average / L

In [82]:
metric = "Xintent"
print("Avedarge deflection of " + metric + " = " + str(float('{:.3f}'.format(check_metrics(metric, test_df)))))
metric = "Xemotion"
print("Avedarge deflection of " + metric + " = " + str(float('{:.3f}'.format(check_metrics(metric, test_df)))))
metric = "Otheremotion"
print("Avedarge deflection of " + metric + " = " + str(float('{:.3f}'.format(check_metrics(metric, test_df)))))

Avedarge deflection of Xintent = 0.205
Avedarge deflection of Xemotion = 0.363
Avedarge deflection of Otheremotion = 0.684
